In [ ]:
import tensorflow as tf
import matplotlib.pyplot as  plt
from tensorflow.keras import models, layers

In [ ]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNEL = 3
EPOCHS = 5

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    
        "PlantVillage",
        shuffle = True,
        image_size = (IMAGE_SIZE, IMAGE_SIZE),
        batch_size = BATCH_SIZE
    )


In [ ]:
class_names = dataset.class_names
class_names


In [ ]:
len(dataset)

In [ ]:
train_size = 0.8 
test_size = 0.1
validation_size = 0.1

len(dataset) * test_size

In [ ]:
train_dataset = dataset.take(604)
len(train_dataset)

In [ ]:
test_dataset = dataset.skip(604)
len(test_dataset)

In [ ]:
val_dataset = test_dataset.take(76)
len(val_dataset)


In [ ]:

test_dataset = test_dataset.skip(76)
len(test_dataset)

In [ ]:
resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.Rescaling(1.0/255)
])

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2)
    
    
])

In [ ]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNEL)
classes_number = 17
model = models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = input_shape ),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size= (3, 3), activation = 'relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, kernel_size= (3, 3), activation = 'relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation= 'relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation= 'relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation = 'relu'),
    layers.Dense(classes_number, activation = 'softmax')
    
    
])

model.build(input_shape = input_shape)

In [ ]:
model.summary() 

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
    metrics = ['accuracy']
)

In [ ]:
history = model.fit(
    train_dataset,
    epochs = EPOCHS,
    batch_size = BATCH_SIZE,
    verbose = 1,
    validation_data = val_dataset
    
)

In [ ]:
scores = model.evaluate(test_dataset)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']


In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
import numpy as np
for images_batch, labels_batch in test_dataset.take(1):
    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()
    
    print("First image to predict")
    plt.imshow(first_image)
    print("actual Disease:", class_names[first_label])
    batch_prediction = model.predict(images_batch)
    print("Predicted Disease:", class_names[np.argmax(batch_prediction[0])])

# Transfer Learning Implementation

Now let's implement transfer learning using pre-trained models to improve validation accuracy. We'll try several popular architectures and compare their performance.


In [18]:
# Import additional modules for transfer learning
from tensorflow.keras.applications import ResNet50, EfficientNetB0, MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import os


In [19]:
# Updated preprocessing for transfer learning
# Most pre-trained models expect inputs in range [0,1] or [-1,1]
resize_and_rescale_tl = tf.keras.Sequential([
    layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.Rescaling(1.0/255)  # Normalize to [0,1]
])

# Enhanced data augmentation for better generalization
data_augmentation_enhanced = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
])

print("Preprocessing layers updated for transfer learning")


Preprocessing layers updated for transfer learning


In [20]:
def create_transfer_learning_model(base_model_name='resnet50', input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNEL), 
                                  num_classes=classes_number, trainable_base=False):
    """
    Create a transfer learning model using pre-trained base models
    
    Args:
        base_model_name: 'resnet50', 'efficientnet', or 'mobilenet'
        input_shape: Input shape for the model
        num_classes: Number of output classes
        trainable_base: Whether to make base model layers trainable
    """
    
    # Load pre-trained base model
    if base_model_name.lower() == 'resnet50':
        base_model = ResNet50(
            weights='imagenet',
            include_top=False,
            input_shape=input_shape
        )
    elif base_model_name.lower() == 'efficientnet':
        base_model = EfficientNetB0(
            weights='imagenet',
            include_top=False,
            input_shape=input_shape
        )
    elif base_model_name.lower() == 'mobilenet':
        base_model = MobileNetV2(
            weights='imagenet',
            include_top=False,
            input_shape=input_shape
        )
    else:
        raise ValueError("Unsupported base model. Choose from: 'resnet50', 'efficientnet', 'mobilenet'")
    
    # Freeze base model layers initially
    base_model.trainable = trainable_base
    
    # Create the complete model
    model = models.Sequential([
        resize_and_rescale_tl,
        data_augmentation_enhanced,
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model, base_model

print("Transfer learning model creation function defined")


Transfer learning model creation function defined


In [21]:
# Create ResNet50 transfer learning model
print("Creating ResNet50 transfer learning model...")
resnet_model, resnet_base = create_transfer_learning_model('resnet50')

# Compile the model
resnet_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

print("ResNet50 model summary:")
resnet_model.summary()


Creating ResNet50 transfer learning model...
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step
ResNet50 model summary:


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_3 (Sequential)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_4 (Sequential)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 8, 8, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ ?                      │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

In [22]:
# Setup callbacks for better training
callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        'best_resnet_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

print("Callbacks configured for optimal training")


Callbacks configured for optimal training


In [ ]:
# Train ResNet50 model with transfer learning
print("Training ResNet50 model with frozen base layers...")
EPOCHS_TL = 15  # More epochs for transfer learning

history_resnet = resnet_model.fit(
    train_dataset,
    epochs=EPOCHS_TL,
    validation_data=val_dataset,
    callbacks=callbacks,
    verbose=1
)

print("ResNet50 training completed!")


In [ ]:
# Fine-tuning: Unfreeze some layers for better performance
print("Starting fine-tuning phase...")

# Unfreeze the top layers of the base model
resnet_base.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = len(resnet_base.layers) - 20

# Freeze all the layers before fine_tune_at
for layer in resnet_base.layers[:fine_tune_at]:
    layer.trainable = False

# Recompile with a lower learning rate
resnet_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001/10),  # Lower learning rate
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

print(f"Fine-tuning from layer {fine_tune_at} onwards")
print(f"Total layers: {len(resnet_base.layers)}")
print(f"Trainable layers: {sum([layer.trainable for layer in resnet_base.layers])}")


In [ ]:
# Fine-tuning training
fine_tune_epochs = 10
total_epochs = len(history_resnet.history['accuracy']) + fine_tune_epochs

# Update callbacks for fine-tuning
callbacks_finetune = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=3,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=1e-8,
        verbose=1
    ),
    ModelCheckpoint(
        'best_resnet_finetuned.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

history_fine = resnet_model.fit(
    train_dataset,
    epochs=total_epochs,
    initial_epoch=len(history_resnet.history['accuracy']),
    validation_data=val_dataset,
    callbacks=callbacks_finetune,
    verbose=1
)

print("Fine-tuning completed!")


In [ ]:
# Evaluate the transfer learning model
print("Evaluating ResNet50 transfer learning model...")
resnet_scores = resnet_model.evaluate(test_dataset, verbose=1)
print(f"ResNet50 Test Accuracy: {resnet_scores[1]:.4f}")
print(f"ResNet50 Test Loss: {resnet_scores[0]:.4f}")


In [ ]:
# Compare Original vs Transfer Learning Models
def plot_training_comparison(original_history, transfer_history, fine_history=None):
    """Plot comparison between original and transfer learning training"""
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Combine transfer learning and fine-tuning histories
    if fine_history:
        tl_acc = transfer_history.history['accuracy'] + fine_history.history['accuracy']
        tl_val_acc = transfer_history.history['val_accuracy'] + fine_history.history['val_accuracy']
        tl_loss = transfer_history.history['loss'] + fine_history.history['loss']
        tl_val_loss = transfer_history.history['val_loss'] + fine_history.history['val_loss']
    else:
        tl_acc = transfer_history.history['accuracy']
        tl_val_acc = transfer_history.history['val_accuracy']
        tl_loss = transfer_history.history['loss']
        tl_val_loss = transfer_history.history['val_loss']
    
    # Training Accuracy Comparison
    axes[0, 0].plot(original_history.history['accuracy'], 'b-', label='Original CNN', linewidth=2)
    axes[0, 0].plot(tl_acc, 'r-', label='ResNet50 Transfer Learning', linewidth=2)
    axes[0, 0].set_title('Training Accuracy Comparison')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Validation Accuracy Comparison
    axes[0, 1].plot(original_history.history['val_accuracy'], 'b-', label='Original CNN', linewidth=2)
    axes[0, 1].plot(tl_val_acc, 'r-', label='ResNet50 Transfer Learning', linewidth=2)
    axes[0, 1].set_title('Validation Accuracy Comparison')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Training Loss Comparison
    axes[1, 0].plot(original_history.history['loss'], 'b-', label='Original CNN', linewidth=2)
    axes[1, 0].plot(tl_loss, 'r-', label='ResNet50 Transfer Learning', linewidth=2)
    axes[1, 0].set_title('Training Loss Comparison')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Loss')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Validation Loss Comparison
    axes[1, 1].plot(original_history.history['val_loss'], 'b-', label='Original CNN', linewidth=2)
    axes[1, 1].plot(tl_val_loss, 'r-', label='ResNet50 Transfer Learning', linewidth=2)
    axes[1, 1].set_title('Validation Loss Comparison')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Loss')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print final metrics comparison
    print("\\n" + "="*50)
    print("FINAL PERFORMANCE COMPARISON")
    print("="*50)
    print(f"Original CNN - Final Val Accuracy: {original_history.history['val_accuracy'][-1]:.4f}")
    print(f"Transfer Learning - Final Val Accuracy: {tl_val_acc[-1]:.4f}")
    print(f"Improvement: {(tl_val_acc[-1] - original_history.history['val_accuracy'][-1]):.4f}")
    print(f"Relative Improvement: {((tl_val_acc[-1] - original_history.history['val_accuracy'][-1]) / original_history.history['val_accuracy'][-1] * 100):.2f}%")

# Plot the comparison (you'll run this after training both models)
# plot_training_comparison(history, history_resnet, history_fine)


In [ ]:
# Alternative Transfer Learning Models
print("Creating alternative transfer learning models for comparison...")

# EfficientNet Model
print("\\n1. Creating EfficientNet model...")
efficientnet_model, efficientnet_base = create_transfer_learning_model('efficientnet')
efficientnet_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# MobileNet Model
print("\\n2. Creating MobileNet model...")
mobilenet_model, mobilenet_base = create_transfer_learning_model('mobilenet')
mobilenet_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

print("\\nAll transfer learning models created successfully!")
print("You can now train these models and compare their performance.")


In [ ]:
# Save the best transfer learning model
print("Saving the best transfer learning model...")
resnet_model.save('plant_disease_resnet50_v2.h5')
print("Model saved as 'plant_disease_resnet50_v2.h5'")

# Test prediction with transfer learning model
print("\\nTesting prediction with transfer learning model...")
for images_batch, labels_batch in test_dataset.take(1):
    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()
    
    print("First image to predict")
    plt.figure(figsize=(8, 6))
    plt.imshow(first_image)
    plt.title(f"Actual Disease: {class_names[first_label]}")
    plt.axis('off')
    plt.show()
    
    # Predictions from both models
    original_pred = model.predict(images_batch)
    transfer_pred = resnet_model.predict(images_batch)
    
    print(f"\\nActual Disease: {class_names[first_label]}")
    print(f"Original CNN Prediction: {class_names[np.argmax(original_pred[0])]} (Confidence: {np.max(original_pred[0]):.3f})")
    print(f"Transfer Learning Prediction: {class_names[np.argmax(transfer_pred[0])]} (Confidence: {np.max(transfer_pred[0]):.3f})")
    
    break


## Transfer Learning Implementation Summary

**What we've implemented:**

1. **Pre-trained Base Models**: ResNet50, EfficientNet, and MobileNet with ImageNet weights
2. **Two-Phase Training**: 
   - Phase 1: Frozen base model layers (transfer learning)
   - Phase 2: Fine-tuning with unfrozen top layers
3. **Enhanced Data Augmentation**: Added zoom and contrast for better generalization
4. **Advanced Callbacks**: Early stopping, learning rate reduction, and model checkpointing
5. **Model Comparison**: Tools to compare original CNN vs transfer learning performance

**Expected Benefits:**
- **Higher Validation Accuracy**: Pre-trained features should significantly improve performance
- **Faster Convergence**: Transfer learning typically requires fewer epochs
- **Better Generalization**: Pre-trained models have learned robust feature representations
- **Reduced Overfitting**: Better feature extraction reduces the need for complex architectures

**Next Steps:**
1. Run the transfer learning training cells above
2. Compare results using the comparison function
3. Choose the best performing model for deployment
4. Update your API to use the new model

**Tips for Best Results:**
- Start with ResNet50 as it's proven effective for image classification
- Monitor validation accuracy - you should see significant improvement
- If overfitting occurs, increase dropout or reduce learning rate
- Consider ensemble methods combining multiple pre-trained models


In [ ]:
model_version = '1'
model.save(f"../models/plant_disease_{model_version}")